# Testing the normal label formulation and the DL Enhancement / Lifted Formulation from https://link.springer.com/content/pdf/10.1007/s13676-012-0010-0.pdf

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp, atsp_DT_formulation, tsp_callbacks
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    print("writing solutions...")
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        
        G = tsp.get_graph()
        G = imp_nx.read(G)
        #m = gen_path_atsp.createGenModel(G,'min','metric')
        #m = atsp_DT_formulation.createGenModel(G, "min", "metric")
        m = tsp_callbacks.createGenModel(G, "min", "metric")
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1
    print(solutions)
    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/p654.tsp"]
    
    runBenchmark(paths)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4965 rows, 2739025 columns and 5479705 nonzeros
Model fingerprint: 0x1a850a53
Variable types: 0 continuous, 2739025 integer (2739025 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
In Callback
Presolve removed 1655 rows and 1655 columns (presolve time = 6s) ...
Presolve removed 1655 rows and 1655 columns
Presolve time: 7.80s
Presolved: 3310 rows, 2737370 columns, 5474740 nonzeros
Variable types: 0 continuous, 2737370 integ

GurobiError: Out of memory

# Heuristic implementation. Using Nearest Neighbour to create a new Tour and 2 Opt to improve the tour

In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
nx.__version__

'2.5'

In [27]:
import sys, os
sys.path.append('../..')
sys.path.append('../../..')
from imports import ilpgraph, readFile
from imports import networkx as imp_nx
from network.heuristics import tsp_nearest_neighbour as NN
from network.heuristics import tsp_two_opt
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt
import tsplib95



# Path of the TSPLib Instance
path = r"../TSPTestInstances/kroB150.tsp"

# Loading problem via tsplib95. Returns a dictionary with a bunch of information that the code doesn't need.
problem = tsplib95.load(path)
G = problem.get_graph()
Graph = imp_nx.read(G)

# Calculate Nearest Neighbours
tour, length = NN.getHeuristic(Graph)
print("Nearest Neighbour Tour length ", length)
print("2 OPT")
# Improve with 2 OPT Heuristic
tour, length = tsp_two_opt.getHeuristic(Graph, tour, length, 150)
print(tour)
print(length)



Nearest Neighbor Tour  [(1, 53), (53, 85), (85, 27), (27, 15), (15, 13), (13, 79), (79, 110), (110, 64), (64, 20), (20, 55), (55, 42), (42, 67), (67, 31), (31, 47), (47, 6), (6, 149), (149, 54), (54, 134), (134, 75), (75, 22), (22, 112), (112, 8), (8, 106), (106, 17), (17, 25), (25, 90), (90, 34), (34, 145), (145, 109), (109, 98), (98, 148), (148, 88), (88, 28), (28, 39), (39, 38), (38, 71), (71, 124), (124, 72), (72, 83), (83, 62), (62, 138), (138, 113), (113, 108), (108, 114), (114, 126), (126, 101), (101, 56), (56, 119), (119, 5), (5, 43), (43, 86), (86, 133), (133, 50), (50, 95), (95, 94), (94, 123), (123, 91), (91, 70), (70, 111), (111, 121), (121, 76), (76, 131), (131, 139), (139, 21), (21, 122), (122, 89), (89, 41), (41, 59), (59, 3), (3, 73), (73, 118), (118, 69), (69, 74), (74, 57), (57, 36), (36, 100), (100, 33), (33, 45), (45, 81), (81, 97), (97, 144), (144, 136), (136, 10), (10, 92), (92, 19), (19, 105), (105, 99), (99, 141), (141, 14), (14, 143), (143, 93), (93, 49), (49, 